1.  Read the CSV file provided into a Spark DataFrame.

2.  Count the number of males and females who had their Cabin as null.

3.  Find the average ages of passengers.

4.  Fill in the missing age values with this average value.

5.  Save the output to a CSV file in HDFS (depi_folder).

6.  Count the total number of passengers who survived and those who did
    not.

7.  Find the top 5 most common embarkation ports among passengers.

8.  Calculate the survival rate by passenger class (Pclass).

9.  Determine the maximum, minimum, and average fare paid by passengers.

10. Write a Spark job to count the number of passengers in each age
    group (e.g., 0–18, 19–35, 36–60, 60+).

11. Create a new directory in HDFS called titanic_lab and list its
    contents.

12. Upload the Titanic dataset from your local machine to the
    titanic_lab directory in HDFS.

13. Use chmod command to change the permissions of the Titanic dataset
    file to 777 (full permissionsfor all)

14. Use-cat to display the first 20 lines of the Titanic dataset stored
    in HDFS.

15- Move the processed output file from titanic_lab to a new directory
in HDFS called titanic_results using hdfs dfs -mv.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TitanicCSVExample") \
    .getOrCreate()

df = spark.read.csv(
    r"E:\Material\Skills Dynamix\Spark,Hadoop_Task\titanic.csv",
    header=True,
    inferSchema=True
)

df.show(10)

df.printSchema()



+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [ ]:
from pyspark.sql.functions import col


null_cabin_df = df.filter(col("Cabin").isNull())


male_count = null_cabin_df.filter(col("gender") == "male").count()
female_count = null_cabin_df.filter(col("gender") == "female").count()

print("Males with null Cabin:", male_count)
print("Females with null Cabin:", female_count)





Males with null Cabin: 470
Females with null Cabin: 217


In [ ]:
from pyspark.sql.functions import avg


avg_age = df.select(avg("Age")).collect()[0][0]

print("Average Age of Passengers:", avg_age)


Average Age of Passengers: 29.69911764705882


In [ ]:
from pyspark.sql.functions import col, when


avg_age = df.select(avg("Age")).collect()[0][0]


df_filled = df.withColumn("Age",
               when(col("Age").isNull(), avg_age).otherwise(col("Age")))

df_filled.show(10)


+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|

In [ ]:
ر
output_path = "hdfs://localhost:9000/depi_folder/titanic_output"

df_filled.write \
    .option("header", True) \
    .mode("overwrite") \
    .csv(output_path)


In [ ]:
!pip install wget
import os
import wget
import tarfile


HADOOP_URL = "https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz"
HADOOP_TAR = "hadoop-3.3.6.tar.gz"
HADOOP_DIR = "C:/hadoop"  

if not os.path.exists(HADOOP_TAR):
    print("Downloading Hadoop...")
    wget.download(HADOOP_URL, HADOOP_TAR)
else:
    print("Hadoop archive already exists.")

if not os.path.exists(HADOOP_DIR):
    print("\nExtracting Hadoop...")
    with tarfile.open(HADOOP_TAR, "r:gz") as tar:
        tar.extractall(path=os.path.dirname(HADOOP_DIR))
    os.rename("hadoop-3.3.6", HADOOP_DIR)
else:
    print("Hadoop directory already exists.")

os.environ["HADOOP_HOME"] = HADOOP_DIR
os.environ["PATH"] += ";" + os.path.join(HADOOP_DIR, "bin")

print("\n Hadoop downloaded and extracted at:", HADOOP_DIR)


In [ ]:
output_path = r"E:\Material\Skills Dynamix\Spark,Hadoop_Task\depi_folder\titanic_output"

df_filled.write \
    .option("header", True) \
    .mode("overwrite") \
    .csv(output_path)


In [ ]:
import pandas as pd


df = pd.read_csv("titanic.csv")


survival_counts = df['Survived'].value_counts()

print(survival_counts)


Survived
0    549
1    342
Name: count, dtype: int64


In [17]:
# 7.  Find the top 5 most common embarkation ports among passengers.
top_ports = df['Embarked'].value_counts().head(5)
print(top_ports)

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


In [18]:
# 8.  Calculate the survival rate by passenger class (Pclass)
survival_rate_by_class = df.groupby('Pclass')['Survived'].mean()
print(survival_rate_by_class)



Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


In [20]:
# 9.  Determine the maximum, minimum, and average fare paid by passengers.
fare_stats = df['Fare'].agg(['max', 'min', 'mean'])
print(fare_stats)

max     512.329200
min       0.000000
mean     32.204208
Name: Fare, dtype: float64


In [23]:
# 10. Write a Spark job to count the number of passengers in each age group (e.g., 0–18, 19–35, 36–60, 60+).
from flask.views import F
age_bins = [0, 18, 35, 60, float('inf')]
age_labels = ['0-18', '19-35', '36-60', '60+']
age_groups = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=False)
age_group_counts = age_groups.value_counts().sort_index()
print(age_group_counts)


Age
0-18     113
19-35    366
36-60    209
60+       26
Name: count, dtype: int64


In [ ]:
# 11. Create a new directory in HDFS called titanic_lab and list its
#     contents.
!hdfs dfs -mkdir -p /titanic_lab
!hdfs dfs -ls /titanic_lab

In [ ]:
# 12. Upload the Titanic dataset from your local machine to the
#     titanic_lab directory in HDFS.
!hdfs dfs -put -f titanic.csv /titanic_lab/
#!/usr/bin/env python3


In [ ]:
# 13. Use chmod command to change the permissions of the Titanic dataset
#     file to 777 (full permissions for all).
!hdfs dfs -chmod 777 /titanic_lab/titanic.csv



In [ ]:
# 14. Use-cat to display the first 20 lines of the Titanic dataset stored
#     in HDFS.
!hdfs dfs -cat /titanic_lab/titanic.csv | head -n 20


In [ ]:
# 15- Move the processed output file from titanic_lab to a new directory
# in HDFS called titanic_results using hdfs dfs -mv.
!hdfs dfs -mkdir -p /titanic_results
!hdfs dfs -mv /titanic_lab/titanic.csv /titanic_results/

